In [2]:
using CSV,DataFrames
data = DataFrame(CSV.read("data/gap-development.tsv",delim='\t', allowmissing=:none, normalizenames=true));
first(data,6)

,ID,Text,Pronoun,Pronoun_offset,A,A_offset,A_coref,B,B_offset,B_coref,URL
,String,String,String,Int64,String,Int64,String,String,Int64,String,String
1,development-1,"Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.",her,274,Cheryl Cassidy,191,TRUE,Pauline,207,FALSE,http://en.wikipedia.org/wiki/List_of_Teachers_(UK_TV_series)_characters
2,development-2,"He grew up in Evanston, Illinois the second oldest of five children including his brothers, Fred and Gordon and sisters, Marge (Peppy) and Marilyn. His high school days were spent at New Trier High School in Winnetka, Illinois. MacKenzie studied with Bernard Leach from 1949 to 1952. His simple, wheel-thrown functional pottery is heavily influenced by the oriental aesthetic of Shoji Hamada and Kanjiro Kawai.",His,284,MacKenzie,228,TRUE,Bernard Leach,251,FALSE,http://en.wikipedia.org/wiki/Warren_MacKenzie
3,development-3,"He had been reelected to Congress, but resigned in 1990 to accept a post as Ambassador to Brazil. De la Sota again ran for governor of C*rdoba in 1991. Defeated by Governor Angeloz by over 15%, this latter setback was significant because it cost De la Sota much of his support within the Justicialist Party (which was flush with victory in the 1991 mid-terms), leading to President Carlos Menem 's endorsement of a separate party list in C*rdoba for the 1993 mid-term elections, and to De la Sota's failure to regain a seat in Congress.",his,265,Angeloz,173,FALSE,De la Sota,246,TRUE,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_de_la_Sota
4,development-4,"The current members of Crime have also performed in San Francisco under the band name ''Remote Viewers``. Strike has published two works of fiction in recent years: Ports of Hell, which is listed in the Rock and Roll Hall of Fame Library, and A Loud Humming Sound Came from Above. Rank has produced numerous films (under his real name, Henry Rosenthal) including the hit The Devil and Daniel Johnston.",his,321,Hell,174,FALSE,Henry Rosenthal,336,TRUE,http://en.wikipedia.org/wiki/Crime_(band)
5,development-5,"Her Santa Fe Opera debut in 2005 was as Nuria in the revised edition of Golijov's Ainadamar. She sang on the subsequent Deutsche Grammophon recording of the opera. For his opera Doctor Atomic, Adams rewrote the role of Kitty Oppenheimer, originally a mezzo-soprano role, for soprano voice, and Rivera sang the rewritten part of Kitty Oppenheimer at Lyric Opera of Chicago, De Nederlandse Opera, and the Metropolitan Opera., all in 2007. She has since sung several parts and roles in John Adams' works, including the soprano part in El Ni*o, and the role of Kumudha in A Flowering Tree in the Peter Sellars production at the New Crowned Hope Festival in Vienna.",She,437,Kitty Oppenheimer,219,FALSE,Rivera,294,TRUE,http://en.wikipedia.org/wiki/Jessica_Rivera
6,development-6,"Sandra Collins is an American DJ. She got her start on the West Coast of the U.S. in Phoenix, Arizona and into residencies in Los Angeles, and eventually moved towards trance. She used American producers to give herself a unique sound. Collins performed for an estimated 80,000 people on the first night of Woodstock '99, and was the first female DJ featured in the Tranceport series of influential recordings. She recently has released two CD mixes under Paul Oakenfold's Perfecto label.",She,411,Collins,236,TRUE,DJ,347,FALSE,http://en.wikipedia.org/wiki/Sandra_Collins


In [2]:
data[:Text]= map(lowercase, data[:Text]);

In [7]:
#using Pkg
#Pkg.add("Embeddings")
using Embeddings
const embtable = load_embeddings(GloVe,4) # or load_embeddings(FastText_Text) or ...


Embeddings.EmbeddingTable{Array{Float32,2},Array{String,1}}(Float32[0.04656 -0.25539 … 0.81451 0.429191; 0.21318 -0.25723 … -0.36221 -0.296897; … ; -0.20989 -0.12226 … 0.28408 0.32618; 0.053913 0.35499 … -0.17559 -0.0590532], ["the", ",", ".", "of", "to", "and", "in", "a", "\"", "'s"  …  "sigarms", "katuna", "aqm", "1.3775", "corythosaurus", "chanty", "kronik", "rolonda", "zsombor", "sandberger"])

In [6]:
const get_word_index = Dict(word=>ii for (ii,word) in enumerate(embtable.vocab))

function get_embedding(word)
    ind = get_word_index[word]
    emb = embtable.embeddings[:,ind]
    return emb
end

get_embedding (generic function with 1 method)

In [7]:
get_word_index["unk"]

201535

In [8]:
#using Pkg
#Pkg.add("WordTokenizers")
using WordTokenizers


In [9]:
function construct_embeddings(tokens)  
   miss = [] 
    i = 0
    embd = DataFrame(zeros(300,1)) 
    for token in tokens
        try
            em = get_embedding(token)
            if(size(em)[1] == 300)
                #println(token)
                i = i +1
                embd[i] = em[:]  
            else
                push!(miss,token)
                #println(token)
            end
            
        catch y
             #println(token)
             push!(miss,token)
              
        end
        
        
    end
    return embd,miss
end

construct_embeddings (generic function with 1 method)

In [10]:
function get_embd_index(X)
    arr = zeros(150,1) # number of max words in each example
    embd = DataFrame(zeros(150,size(X)[1]))
    for i in 1:10 # number of training sets
        split_text = tokenize.(split_sentences(X[:Text][i]))
        doc = vcat(split_text...)
        #println(length(doc))
        j = 0
        for token in doc
            j = j + 1
            if j < 150
                try
                arr[j] = get_word_index[token]
                catch y
                    
                end
            end
        end
        embd[i] = arr[:]
    end
   return embd     
    
end

get_embd_index (generic function with 1 method)

In [6]:
#using Pkg
#Pkg.add("PaddedViews")
using PaddedViews
using Flux:onehot
function construct_training_data(X,n)
    Train_X = Array{Float64}(undef, 300,150,n)
    Train_Y = Array{Float64}(undef,2,3)
    for i in 1:n
        split_text = tokenize.(split_sentences(X[:Text][i]))
        doc = vcat(split_text...)
        train_x,miss = construct_embeddings(doc)
        z = PaddedView(0.0, convert(Array,train_x), (300, 150))
        Train_X[:,:,i] = z
        #Train_Y[:,i] = onehot(X[:A_coref][i],["TRUE","FALSE"])
        
    end
    Train_Y = onehotbatch(data[:A_coref][1:3], ["TRUE","FALSE"])
    return Train_X,Train_Y
end

construct_training_data (generic function with 1 method)

In [98]:
using Flux:onehotbatch
train_X,train_Y = construct_training_data(data,3)
data_XY = zip(train_X,train_Y)

┌ Warning: `convert(::Type{Array}, df::AbstractDataFrame)` is deprecated, use `convert(Matrix, df)` instead.
│   caller = construct_training_data(::DataFrame, ::Int64) at In[97]:12
└ @ Main .\In[97]:12


Base.Iterators.Zip2{Array{Float64,3},Array{Float64,2}}([0.35212 -0.27052 … 0.0 0.0; -0.18099 -0.19328 … 0.0 0.0; … ; -0.30465 0.2016 … 0.0 0.0; 0.35556 -0.029135 … 0.0 0.0]

[-0.52057 -0.7947 … 0.0 0.0; 0.49562 -0.16287 … 0.0 0.0; … ; -0.22204 -0.022498 … 0.0 0.0; 0.14277 0.016073 … 0.0 0.0]

[-0.52057 -0.32601 … 0.0 0.0; 0.49562 0.25829 … 0.0 0.0; … ; -0.22204 -0.33391 … 0.0 0.0; 0.14277 -0.057473 … 0.0 0.0], [1.0 1.0 0.0; 0.0 0.0 1.0])

DimensionMismatch: DimensionMismatch("dimensions must match")

In [109]:
using Flux;
using Flux:crossentropy
model1 = Chain(   
  LSTM(300,150),
    Dense(150,2,σ),
    Dense(2,1),
    x -> σ.(x))
  
function loss(x,y)
    l = Flux.mse(model1(x), y)
    Flux.reset!(model1)
    l
end




loss (generic function with 1 method)

In [70]:
summary(model1)

"Chain{Tuple{Flux.Recur{Flux.LSTMCell{TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}}},Dense{typeof(identity),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}},Dense{typeof(identity),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}},getfield(Main, Symbol(\"##17#18\"))}}"

In [40]:
summary(train_X)

"300×150×3 Array{Float64,3}"

In [110]:
Flux.train!(loss, params(model1),data_XY, ADAM(0.01))

In [112]:
model1(train_X[:,:,1])

Tracked 1×150 Array{Float32,2}:
 0.613348  0.609959  0.558484  …  0.57889  0.57889  0.57889  0.57889

In [111]:
params(model1)

Params([Float32[0.0131531 -0.0406905 … -0.080676 -0.00139121; 0.0629764 -0.00895918 … 0.11993 -0.0468441; … ; 0.0240156 -0.0376131 … -0.0669475 -0.0587887; -0.107754 0.0265195 … -0.0503924 0.0383268] (tracked), Float32[-0.0636966 0.00390379 … 0.0394069 -0.073785; -0.0369835 0.0635294 … -0.0422509 0.0174551; … ; -0.0233187 0.0497063 … -0.0936188 -0.0577894; 0.0499861 -0.0268253 … -0.00870082 -0.0195944] (tracked), Float32[-0.0275465, -0.0929646, -0.0691896, -0.0216121, 0.0799584, -0.0741766, 0.078809, -0.0390927, 0.0590757, 0.020554  …  -0.0613883, 0.0650747, -0.0607477, -0.0989734, 0.0254741, -0.054243, -0.0678311, -0.0306431, -0.0921476, -0.114982] (tracked), Float32[-0.010331, -0.00778698, -0.00653664, 0.010686, -0.00750533, -0.00534056, -0.0101283, -0.0140193, 0.00734552, 0.0099683  …  -0.0123925, 0.0117347, 0.00684113, -0.010687, -0.00884075, 0.00843259, -0.0102125, -0.00935017, -0.0115397, 0.00859825] (tracked), Float32[-0.0222748, 7.31847e-5, -5.99506e-5, -0.00152807, 0.00557642,